# Introduction

This notebook will select models using two different methods.  

**Method 1**

The best VAMP scores at a given lag and fixed number of implied processes. 

**Method 2**

The best joint ranked VAMP and timescale gap. 



In [1]:
# Method 1